# Requirements:
- Define a new type called StudentProfile, whose instances should encapsulate the following attributes:
  - the student's name
  - the student's GRE score (integers between 130 and 340), and
  - the student's SAT score (integers between 400 and 1600)
- StudentProfile instances should have a customized representation
- The score fields should be validated for the correct type and value, i.e. they should be ints that fall in the
expected range
- If a score field is not specified at instantiation, it must default to the minimum of its respective valid range
- Use descriptors with instance-specific storage to implement these validations
- As an extra challenge, try to maximize code reuse by writing a single general descriptor

In [2]:
class ValidScore:

    def __init__(self, min_score, max_score):
        self.min_score = min_score
        self.max_score = max_score
    
    def __set_name__(self, owner, name):
        self.name = name
        
    def __get__(self, instance, owner):
        if instance is None:
            return self
        return instance.__dict__.get(f"{type(self).__name__}_{self.name}")

    def __set__(self, instance, value):
        if not isinstance(value, int):
            raise TypeError(f"{value} must be int")

        if value < self.min_score or value > self.max_score:
            raise ValueError(f"{value} Score must fall between {self.min_score} and {self.max_score}")

        instance.__dict__[f"{type(self).__name__}_{self.name}"] = value

    def __delete__(self, instance):
        del instance.__dict__[f"{type(self).__name__}_{self.name}"]


class SATScore(ValidScore):

    def __init__(self, min_score = 400, max_score = 1600):
        super().__init__(min_score, max_score)

class GREScore(ValidScore):

    def __init__(self, min_score = 130, max_score = 340):
        super().__init__(min_score, max_score)

In [3]:
class StudentProfile:

    sat = SATScore()
    gre = GREScore()
    
    def __init__(self, name, sat=sat.min_score, gre=gre.min_score):
        self.name = name
        self.sat = sat
        self.gre = gre

    def __repr__(self):
        return f"{type(self).__name__}(name={self.name!r}, sat={self.sat}, gre={self.gre})"

In [4]:
sp = StudentProfile(name="Andrew", sat=1220, gre=130)
sp

StudentProfile(name='Andrew', sat=1220, gre=130)

In [5]:
sp.__dict__

{'name': 'Andrew', 'SATScore_sat': 1220, 'GREScore_gre': 130}

In [6]:
sp2 = StudentProfile("Liza", gre=190)
sp2

StudentProfile(name='Liza', sat=400, gre=190)

In [7]:
try:
    sp2.gre = 2000
except ValueError as err:
    print(err)

2000 Score must fall between 130 and 340


In [8]:
try:
    sp2.gre = 1200.2
except TypeError as err:
    print(err)

1200.2 must be int
